In [11]:
from PIL import Image
from math import sqrt
from math import log
from math import fabs
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

## Transformer la video en plusieurs Frame

In [ ]:
# lire la video dans un fichier 
cap = cv2.VideoCapture('test1.mp4')

try:
    if not os.path.exists('data'):
        os.makedirs('data')
except OSError:
    print ('Error: Creating directory of data')
c=0
currentFrame = 0
currentFrame1= 0
currentFrame2= 0
while(True):
    # Capturer frame par frame
    ret, frame = cap.read()

    # Enregistre l'image du frame actuelle dans un fichier jpg
    name = './data/frame' + str(currentFrame) + str(currentFrame1) + str(currentFrame2) + '.jpg'
    print ('Creating...' + name)
    cv2.imwrite(name, frame)
    # Enregistre l'image du frame actuelle dans notre dossier pour lire les frames jpg
    name = './frame' + str(currentFrame) + str(currentFrame1) + str(currentFrame2) + '.jpg'
    print ('Creating...' + name)
    cv2.imwrite(name, frame)
    # Pour arrêter les images en double
    if currentFrame2 < 9:
        currentFrame2 += 1
    else:
        currentFrame2=0
        currentFrame1+=1
        c=1
    if currentFrame1 < 9 and c==1:
         c=0
    elif c==1:
        currentFrame1=0
        currentFrame+=1
        c=0
# Lorsque tout est terminé, relâchez la capture
cap.release()
cv2.destroyAllWindows()

## mettre les frames de video en ordre dans une liste

In [13]:
data=[]
for filename in os.listdir(f'data'):
    data.append(filename)
 # mettre les frames dans une liste 

## Fonction qui construire l'histogramme de notre frames

In [14]:
def buildHistogram(im):
    pix = im.load()
    if (im.mode == "RGB"):
        R = [0]*256
        G = [0]*256
        B = [0]*256
        histo1=[]
        for i in range(0, im.size[0]):
            for j in range(0, im.size[1]):
                val = pix[i, j]
                R[val[0]] += 1
                G[val[1]] += 1
                B[val[2]] += 1
        histo = (R, G, B)
        return histo;
    elif im.mode == "HSV":
        H = [0]*360
        S = [0]*100
        V = [0]*100
        for i in range(0, im.size[0]):
            for j in range(0, im.size[1]):
                val = pix[i, j]
                H[val[0]] += 1
                S[val[1]] += 1
                V[val[2]] += 1
        histo = (H, S, V)
        return histo;
    else:
        print("Votre image n'est pas en RGB ou en HSV")
    

## Fonction qui normalise les histogrammes

In [15]:
def normHisto(histo):
    r = []
    s = 0.0
    for h in histo:
        s += h
    for h in histo:
        r.append(h/s)
    return r

## fonction qui calcule la distance de Bhattacharyya entre deux histogrammes

In [16]:
def comparaisonHistoBhattacharyya(histoA, histoB):
    hA = normHisto(histoA)   # normaliser l'histogramme A
    hB = normHisto(histoB)   # normaliser l'histogramme B
    distance = 0.
    for i in range(0, 255):
        distance += (hA[i] * hB[i]) ** 0.5
    newDistance = -log(distance)   
    # calculer la distance de Bhattacharyya 
    return newDistance

## fonction qui compare les histogrammes de deux images

In [17]:
def comparaisonHisto3channels(histoA, histoB):
    resultsBatta = []
    moyenneBatta = 0
    for c in range(3):
        # le resultat du calcule de la distance de Bhattacharyya entre histoA[c] et histoB[c]
        resultsBatta.append(comparaisonHistoBhattacharyya(histoA[c], histoB[c])) 
    for i in range(0, 3):
        moyenneBatta += resultsBatta[i]
    moyenneBatta /= 3.               # calculer la moyenne des 3 distances de Bhattacharyya  
    r=round((moyenneBatta * 100), 2)
    return r

## fonction qui compare les histogrammes de deux frames succesives de la video et donner le resulat entre deux frames ( est ce qu'il ya un changement de plan ou non)

In [18]:
def execution():
    for i in range(len(data)):
        histo1=buildHistogram(Image.open(data[i])) #construire l'histogramme de l'image 
        histo2=buildHistogram(Image.open(data[i+1])) #construire l'histogramme de l'image suivante
        compahisto=comparaisonHisto3channels(histo1,histo2) # comparer les histogrammes de deux images successives
        
        if compahisto >= 1:
            print("détection de changement du plan entre frame",i,"et frame",i+1)
        else:
            print("pas de changement de plan entre frame",i,"et frame",i+1)
    return 0

# détection de changement du plan

In [ ]:
execution()